#### PySpark_RealTime_Transforms_Practical.py
#### Practical PySpark script demonstrating a wide variety of DataFrame transformations
#### Includes: local dataset + schema, batch transforms, window functions, UDFs, joins, pivots, union, cache, repartition
#### Also includes a small Structured Streaming example (rate source) that joins streaming events to static reference data.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, TimestampType, ArrayType, MapType
from pyspark.sql.functions import col, expr, when, lit, udf, explode, split, concat_ws, row_number, dense_rank, lag, lead
from pyspark.sql.functions import sum as _sum, avg as _avg, count as _count, max as _max, min as _min
from pyspark.sql.window import Window
import pyspark.sql.functions as F

#### -----------------------------------------------------------------------------
#### Setup SparkSession (local)
#### -----------------------------------------------------------------------------

In [2]:
spark = (SparkSession.builder
         .appName("PySpark_Transforms_Practical")
         .master("local[1]")
         .config("spark.sql.shuffle.partitions", "2")
         .config("spark.python.worker.reuse", "false")
         .config("spark.sql.execution.pyspark.udf.faulthandler.enabled", "true")
         .config("spark.sql.adaptive.enabled", "false")
         .getOrCreate())
spark.sparkContext.setLogLevel("WARN")

#### -----------------------------------------------------------------------------
#### Create a local dataset and schema (static / reference data)
#### -----------------------------------------------------------------------------

In [3]:
customers_schema = StructType([
StructField("customer_id", IntegerType(), False),
StructField("name", StringType(), True),
StructField("city", StringType(), True),
StructField("tier", StringType(), True)
])


customers_data = [
(1, "Asha", "Bengaluru", "Gold"),
(2, "Ravi", "Mumbai", "Silver"),
(3, "Meera", "Delhi", "Gold"),
(4, "Arun", "Chennai", None),
(5, "Sita", "Kolkata", "Bronze")
]

# Transactions dataset (batch example)
transactions_schema = StructType([
StructField("tx_id", IntegerType(), False),
StructField("customer_id", IntegerType(), True),
StructField("amount", DoubleType(), True),
StructField("items", ArrayType(StringType()), True),
StructField("tx_time", TimestampType(), True)
])


transactions_data = [
(101, 1, 250.0, ["pen", "notebook"], None),
(102, 2, 1200.0, ["headphones"], None),
(103, 1, 75.5, ["pencil"], None),
(104, 3, 560.0, ["bag", "wallet"], None),
(105, 5, None, [], None),
(106, 4, 45.0, ["snacks"], None),
(107, 2, 3000.0, ["laptop", "mouse"], None),
(108, 3, 130.0, ["book"], None)
]

transactions_df = spark.createDataFrame(transactions_data, transactions_schema)
transactions_df.show(5)

customers_df = spark.createDataFrame(customers_data, schema=customers_schema)
customers_df.show(5)

Py4JJavaError: An error occurred while calling o50.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (LAPTOP-4K4DB9AG executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed). Consider setting 'spark.sql.execution.pyspark.udf.faulthandler.enabled' or'spark.python.worker.faulthandler.enabled' configuration to 'true' for the better Python traceback.
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:621)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:599)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:35)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:945)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:925)
	at org.apache.

#### create DataFrame (Spark will set null for None in timestamp/amount)

In [ ]:
print("Transactions Data:")
for row in transactions_data[:5]:
    print(row)
print("\nCustomers Data:")
for row in customers_data[:5]:
    print(row)

#### -----------------------------------------------------------------------------
#### Basic transformations (select, filter, where, withColumn, drop)
#### -----------------------------------------------------------------------------

In [ ]:
# select specific columns and expressions